In [1]:
import os
os.chdir("..")

In [2]:
import django
# In case that we need it later
# from django.conf import settings
os.environ.setdefault('DJANGO_SETTINGS_MODULE',
'mimuse.settings')
# This is for async, in case we will see it later (maybe)
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()


In [3]:
import json
import http.client
from songs.models import Song, Album
from artists.models import Artist
from datetime import datetime

In [4]:
Song.objects.all().delete()

(0, {})

In [71]:
import http.client

conn = http.client.HTTPSConnection("spotify23.p.rapidapi.com")

headers = {
    'X-RapidAPI-Key': "82ce6fe0f2msh1b1cfd285dbcc80p1b3260jsn6c515c923b86",
    'X-RapidAPI-Host': "spotify23.p.rapidapi.com"
}

conn.request("GET", "/recommendations/?limit=20&seed_tracks=38MKW2tQHtyO8djIOKlEFF", headers=headers)

res = conn.getresponse()
data = json.loads(res.read().decode("utf-8"))

# print(data.decode("utf-8"))

In [72]:
song_list = list()

track = data['tracks']
for song in data['tracks']:
    # print(song['album']['release_date'])
    if(song['album']['release_date_precision']=="day"):
        song_list.append(Song(api_id=song['id'], title = song['name'], preview_url= song['preview_url'], release_date= song['album']['release_date'], popularity=song['popularity']))
    else:
        song_list.append(Song(api_id=song['id'], title = song['name'], preview_url= song['preview_url'], popularity=song['popularity']))
    # print (song)
# print(song_list)
bulk = Song.objects.bulk_create(song_list)

In [ ]:
for song in data['tracks']:
    print(song)
    id = str(song['artists'][0]['id'])
    song_id = song['id']
    print(song_id)
    if Artist.objects.filter(api_id=id).exists():
        print('Developer msg: data already exists in database')
        continue
    else:
        conn.request("GET", "/artists/?ids={id}".format(
                        id=id), headers=headers)
        res = conn.getresponse()
        datas = json.loads(res.read().decode('utf-8'))['artists'][0]
        artist = Artist(api_id=id, name = datas['name'], popularity=datas['popularity'], image=datas['images'][0]['url'], followers=datas['followers']['total'])
        artist.save()
        song_id = song['id']
        this_song = Song.objects.get(api_id=song_id)
        print(this_song)
        artist.songs.add(this_song)
        artist.save()



In [75]:
Song.objects.get(api_id='2CxgY4VvTNA0bG2nVtmgPg')


<Song: Song object (51)>

In [ ]:
for artist in Artist.objects.all():
    print(artist.songs)

In [ ]:
## get artist singles
for artist in Artist.objects.all():
    api_id = artist.api_id

    conn = http.client.HTTPSConnection("spotify23.p.rapidapi.com")

    headers = {
        'X-RapidAPI-Key': "82ce6fe0f2msh1b1cfd285dbcc80p1b3260jsn6c515c923b86",
        'X-RapidAPI-Host': "spotify23.p.rapidapi.com"
    }

    conn.request("GET", "/artist_singles/?id={api_id}".format(api_id=api_id), headers=headers)

    res = conn.getresponse()
    singles = json.loads(res.read().decode('utf-8'))['data']['artist']['discography']['singles']['items']
    for single in singles 
        id = single['releases']['items'][0]['id']
        if Song.objects.filter(api_id=id).exists():
            continue
        else:
            song_record = Song(api_id=id)
            conn.request("GET", "/tracks/?ids=id".format(id=id), headers=headers)

            res = conn.getresponse()
            song_info = json.loads(res.read().decode("utf-8"))['tracks'][0]
            song_record.save()
            song_artist = Artist.objects.get(api_id=api_id)
            song_artist.songs.add(song_record)
            song_artist.save()
